# 配置环境

在运行之前需要安装所需的python包。建议使用anaconda创建一个新的虚拟环境，具体方法为打开Anaconda navigator，创建环境，环境名称可以命名为ByteBites。

激活环境，确保在vscode的资源管理器中打开ByteBites文件夹。在vscode的左侧python扩展中，在global environments，选中刚才安装的环境，点击open in terminal，在打开的命令行中输入以下命令安装所需的包：
```bash
pip install -r italy/requirements.txt
```

In [2]:
import pandas as pd
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

DATASET_PATH = "..\Amap-results_NJU-Gulou-3000m.csv"

<>:15: SyntaxWarning: invalid escape sequence '\A'
<>:15: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Chrom\AppData\Local\Temp\ipykernel_23772\199738631.py:15: SyntaxWarning: invalid escape sequence '\A'
  DATASET_PATH = "..\Amap-results_NJU-Gulou-3000m.csv"


# 导入CSV数据库

以下代码从CSV文件导入餐厅数据，并生成一个新的文档对象documents。这个功能由DataFrameLoader提供。

### 数据结构说明

documents可以理解为一个list表格，每一行是包含两个字段的dict，两个字段分别为：  
- `page_content`：值为字符串。将餐厅的所有信息整合在一个字符串中，用于喂给大模型。
- `metadata`：值为dict。储存准备用于初过滤的硬性指标值，如经纬度、营业时间等。 

### 示例

假设原始csv文件如下：
```csv
name,address,location,type,tel,cost,rating,opentime_today,opentime_week,tag
巴蜀鱼花(南大店),湖南路街道汉口路30号,"118.779220,32.053685",餐饮服务;中餐厅;火锅店,15380870767,,4.4,09:00-21:00,周一至周日 09:00-21:00,
陕老顺肉夹馍,汉口路30号,"118.779105,32.053716",餐饮服务;餐饮相关场所;餐饮相关,15924140124,,4.4,10:00-21:00,周一至周日 10:00-21:00,肉夹馍
```

导入后的结果为：
```python
documents = [
    ..., # 第一行略
    {
        "page_content": "name=陕老顺肉夹馍\naddress=汉口路30号\nlocation=118.779105,32.053716\ntype=餐饮服务;餐饮相关场所;餐饮相关\ntag=肉夹馍\nrating=4.4\nopentime_today=10:00-21:00\nopentime_week=周一至周日 10:00-21:00\ntel=15924140124",
        "metadata": {
            "location": "118.779105,32.053716",
            "opentime_week": "周一至周日 10:00-21:00"
        }
    }
]
```

### **注意**

- 读取店铺数据的代码更改，导致csv文件中字段发生变化时：def content_func 中 python content_fields 的值要随之更改。
- 想作为硬性指标的字段发生变化时：调用部分 metadata_fields 中的值要更改。

In [ ]:
def get_documents(content_func=lambda row: row['name'] + '\n' + row['tag'],  # 用于导入page_content的函数。如果调用时没传这个参数，默认是将每一行的name和tag拼接起来。
                  # source_func=lambda row: row['address'],  # 后面没用到这个函数。用于导入source的函数。默认是将address作为文档来源。
                  metadata_fields=[]):  # 导入metadata（一个dict）时打算加入进去的字段列表，默认为空。

    # 加载数据库，读取指定路径的 CSV 文件
    dataset_df = pd.read_csv(DATASET_PATH)
    dataset_df.drop_duplicates(inplace=True) # 删除重复数据，确保数据唯一性

    # 对原有数据分别用函数content_func和source_func处理，添加新的列 'page_content' 和'source'，这两列的值由两个函数得到
    dataset_df['page_content'] = dataset_df.apply(content_func, axis=1)
    # dataset_df['source'] = dataset_df.apply(source_func, axis=1)

    # 将 'page_content' 字段添加到metadata_fields中
    metadata_fields = list(set(metadata_fields + ['page_content']))

    # 使用 DataFrameLoader 生成一个新文档对象。 'page_content'列作为文档内容，其他所有列都作为metadata的内容
    loader = DataFrameLoader(dataset_df[metadata_fields], page_content_column='page_content')
    return loader.load()

def content_func(row) -> str: # 定义content_func函数，用于把每家店的所有信息拼到一起，返回成一个字符串。中间用换行符隔开。
  content_fields = ["name",
                    "address",
                    # "location", 
                    "type",
                    "tag",
                    "cost",
                    "rating",
                    "opentime_today",
                    "opentime_week", 
                    # "tel"
                    ]
  return '\n'.join(f"{key}={row[key]}" for key in content_fields if pd.notna(row[key]))

# 调用
metadata_fields = ["location", "opentime_week"]
documents = get_documents(content_func, metadata_fields=metadata_fields)

# 展示
print(documents[1].page_content)
print(documents[1].metadata)

name=陕老顺肉夹馍
address=汉口路30号
location=118.779105,32.053716
type=餐饮服务;餐饮相关场所;餐饮相关
tag=肉夹馍
rating=4.4
opentime_today=10:00-21:00
opentime_week=周一至周日 10:00-21:00
tel=15924140124
{'location': '118.779105,32.053716', 'opentime_week': '周一至周日 10:00-21:00'}


In [ ]:
# from dotenv import load_dotenv
# from langchain_openai import OpenAIEmbeddings
# import os

# #加载环境变量
# load_dotenv()
# #配置嵌入模型
# EMBEDDING_MODEL_NAME = "text-embedding-3-small"  
# embedding_model = OpenAIEmbeddings(
#     model=EMBEDDING_MODEL_NAME,
#     openai_api_key=os.getenv("OPENAI_API_KEY")
# )

# 配置嵌入vecterbase所用的模型

用的是huggingface上面的某个轻量级开源模型。

In [8]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
import os
import numpy as np

# 初始化 HuggingFaceEmbeddings 模型
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # 轻量级模型
    model_kwargs={"device": "cpu"},  # 如果电脑上安装了英伟达显卡，可改成 "cuda" 来加速
    encode_kwargs={"normalize_embeddings": True})  # 归一化

# 生成查询的嵌入向量
result = embedding_model.embed_query("查询向量示例")

# 将结果转换为 numpy 数组并打印信息
array = np.array(result)
print(f"embedding shape: {array.shape}\nembedding norm: {np.linalg.norm(array, ord=2)}")


embedding shape: (384,)
embedding norm: 1.0000000653766763


# FAISS数据库

在代码 FAISS_REVIEWS_PATH_COSINE = "faiss_index_cosine" 中，"faiss_index_cosine" 是 FAISS 向量索引的本地存储路径，保存的是经过向量化处理后的文档索引数据。以下是详细解释：

存储的具体内容
当调用 vector_db.save_local(FAISS_REVIEWS_PATH_COSINE) 时，会在该路径下生成以下文件：

index.faiss
二进制文件，存储向量索引的核心数据（包括向量数据、索引结构等）。

index.pkl（可选）
存储元数据（如文档的原始文本、ID等，需通过 LangChain 额外配置）。

这些文件共同构成一个完整的可复用的向量数据库。

In [ ]:
FAISS_REVIEWS_PATH_COSINE = "faiss_index_cosine" # 向量库存储路径
FAISS_INDEX_NAME = "index" # 向量库索引名称
FAISS_DISTANCE_STRATEGY_COSINE = "COSINE_DISTANCE" # 向量库距离计算策略

# 用于根据csv数据生成向量库的函数。documents就是前面csv数据的导入结果。embedding_model就是上面定义的嵌入模型。
def get_vector_database(documents, embedding_model, distance_strategy):

  vector_database = FAISS.from_documents(
      documents, embedding_model,
      distance_strategy= distance_strategy
      )
  return vector_database

# 嵌入向量库。分批次处理文档，加入了等待时间，避免API限制（现在的版本是把向量库保存在本地，没有对应限制）。
import time
doclen = len(documents) # 这里的长度指的是前面读的数据的行数。
for batch in range(doclen//100 + 1): # 将每个店铺的信息独立转换为一个向量，且每次并行处理 100 个店铺的向量
    docs = documents[batch*100:(batch+1)*100]
    if batch == 0:
        vector_db = get_vector_database(docs, embedding_model, FAISS_DISTANCE_STRATEGY_COSINE)
    else:
        vector_db.merge_from(get_vector_database(docs, embedding_model, FAISS_DISTANCE_STRATEGY_COSINE))
    time.sleep(10) # 每次处理完100条数据，休眠10秒，防止api限制。
    
#储存并加载向量库
vector_db.save_local(folder_path=FAISS_REVIEWS_PATH_COSINE, index_name=FAISS_INDEX_NAME)
vector_db = FAISS.load_local(folder_path=FAISS_REVIEWS_PATH_COSINE,
                             embeddings=embedding_model,
                             index_name=FAISS_INDEX_NAME,
                             allow_dangerous_deserialization=True) # 允许反序列化

### 验证效果

In [ ]:
docs = vector_db.similarity_search("馄饨，汉口路", k = 5)
for doc in docs:
    print(doc, end="\n\n")

page_content='name=箪食记(汉口路店)
address=汉口路75-2号
location=118.777185,32.053960
type=餐饮服务;中餐厅;中餐厅
cost=31.0
rating=4.2
opentime_week=周一至周六 09:00-20:00
tel=13951883009;15651837225' metadata={'location': '118.777185,32.053960', 'opentime_week': '周一至周六 09:00-20:00'}

page_content='name=膳当家黄焖鸡米饭(汉口路店)
address=汉口路38号
location=118.778647,32.053664
type=餐饮服务;中餐厅;中餐厅
tag=鸡肉
cost=19.0
rating=4.4
opentime_today=09:00-21:00
opentime_week=周一至周日 09:00-21:00
tel=13851644907;15305150776' metadata={'location': '118.778647,32.053664', 'opentime_week': '周一至周日 09:00-21:00'}

page_content='name=沙县小吃(汉口路店)
address=汉口路73号
location=118.778214,32.053989
type=餐饮服务;中餐厅;特色/地方风味餐厅
cost=13.0
rating=4.4
opentime_today=10:00-21:00
opentime_week=周一至周日 10:00-21:00
tel=18752023233;19941534156' metadata={'location': '118.778214,32.053989', 'opentime_week': '周一至周日 10:00-21:00'}

page_content='name=鱼塘鲜专业鱼馆(汉口路店)
address=华侨路街道汉口路32号
location=118.778987,32.053660
type=餐饮服务;中餐厅;海鲜酒楼
cost=66.0
rating=4.4
opentime_today=09:00-21:3

In [22]:
docs = vector_db.similarity_search("披萨", k = 5)
for doc in docs:
    print(doc, end="\n\n")

page_content='name=台式烩饭(金银街小区店)
address=上海路158号1栋2单元
location=118.774736,32.057582
type=餐饮服务;中餐厅;台湾菜
cost=17.0
rating=4.4
opentime_today=09:00-22:00
opentime_week=周一至周日 09:00-22:00
tel=025-83308211' metadata={'location': '118.774736,32.057582', 'opentime_week': '周一至周日 09:00-22:00'}

page_content='name=鱼你在一起(恒基中心公寓店)
address=丹凤街29号大润发一层
location=118.786668,32.054497
type=餐饮服务;餐饮相关场所;餐饮相关
rating=4.2
opentime_today=09:00-21:00
opentime_week=周一至周日 09:00-21:00
tel=13770909712' metadata={'location': '118.786668,32.054497', 'opentime_week': '周一至周日 09:00-21:00'}

page_content='name=陕老顺肉夹馍
address=汉口路30号
location=118.779105,32.053716
type=餐饮服务;餐饮相关场所;餐饮相关
tag=肉夹馍
rating=4.4
opentime_today=10:00-21:00
opentime_week=周一至周日 10:00-21:00
tel=15924140124' metadata={'location': '118.779105,32.053716', 'opentime_week': '周一至周日 10:00-21:00'}

page_content='name=美鸽记·中山石岐乳鸽
address=青岛路33-3号
location=118.778138,32.052878
type=餐饮服务;中餐厅;中餐厅
tag=红米肠
rating=4.6
opentime_today=10:00-14:00 16:00-22:00
opentime_wee

# 导入大语言模型

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceHub
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import getpass

# 加载环境变量（包括模型配置）
load_dotenv()

# ========== 可配置部分（支持环境变量或直接修改此处）==========
client = OpenAI(
  api_key=os.getenv("QWEN_API_KEY"),
  base_url=os.getenv("QWEN_BASE_URL")
)
MODEL = os.getenv("QWEN_MODEL_NAME") # qwen2.5-14b-instruct-1m
MODEL_CONFIG = {
    "provider": os.getenv("MODEL_PROVIDER", "openai"),  # 可选：openai / huggingface / local
    "model_name": os.getenv("MODEL_NAME", "gpt-4o"),     # 模型名称
    "api_key": os.getenv("API_KEY", ""),                 # 各平台的API Key
    "local_model_path": os.getenv("LOCAL_MODEL_PATH", "")# 本地模型路径（如果 provider=local）
}
# =====================================================

def init_llm(config: dict) -> object:
    """根据配置初始化大模型"""
    provider = config["provider"].lower()
    
    if provider == "openai":
        if not config["api_key"]:
            config["api_key"] = getpass.getpass("Enter OpenAI API Key: ")
        return ChatOpenAI(
            model=config["model_name"],
            openai_api_key=config["api_key"]
        )
    elif provider == "huggingface":
        if not config["api_key"]:
            config["api_key"] = getpass.getpass("Enter HuggingFace Hub Token: ")
        return HuggingFaceHub(
            repo_id=config["model_name"],
            huggingfacehub_api_token=config["api_key"]
        )
    elif provider == "local":
        from langchain_community.llms import LlamaCpp  # 示例：本地运行 Llama
        return LlamaCpp(
            model_path=config["local_model_path"],
            temperature=0.5
        )
    else:
        raise ValueError(f"不支持的模型提供商: {provider}")

# 初始化模型
llm = init_llm(MODEL_CONFIG)

# 测试调用
messages = [
    SystemMessage("Translate the following from English into French"),
    HumanMessage("hi!"),
]
response = llm.invoke(messages)
print(response.content)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: p. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## LangChain pipeline

In [25]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
review_template_str = """
你的工作是使用高德地图上的餐馆和酒吧评论来帮助人们找到吃饭或喝酒的最佳地点。
使用以下信息和评论回答问题。如果上下文不是关于餐馆的，
然后请告诉用户，您只能提供帮助并回答与餐厅相关的问题。
如果你根据上下文不知道答案，就说你不知道。答案上下文
{context}
"""

system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_template_str
    )
)

human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [system_prompt, human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

reviews_retriever = vector_db.as_retriever(search_kwargs={'k': 20,})

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | llm
    | StrOutputParser()
)

## 案例应用

In [28]:
question = """汉口路哪里可以吃到四川的面"""
print(review_chain.invoke(question))

在汉口路，您可以尝试以下餐馆，提供四川菜和面食：

1. **四川宜宾燃面**
   - 地址：汉口路48号
   - 评分：4.7
   - 营业时间：周一至周日 09:00-20:30
   - 电话：13072527682; 17326125923

2. **兰州拉面刀削面(汉口路店)**
   - 地址：汉口路36号
   - 评分：4.5
   - 营业时间：周一至周日 09:00-21:00
   - 电话：17327791285

3. **西安特色面馆(汉口路店)**
   - 地址：汉口路47号01幢一楼
   - 评分：4.0
   - 营业时间：每天 07:20-21:30
   - 电话：18114808698

这些地方都可以尝到美味的四川面。希望您能享用美食！


In [31]:
question = """我想吃肯德基"""
print(review_chain.invoke(question))

你可以去以下两个肯德基：

1. **肯德基(珠江路地铁店)**
   - 地址：中山路221号负一层101至106室115至119室
   - 营业时间：周一至周日 06:00-21:00
   - 电话：025-85601286
   - 评分：4.3
   - 费用：约21元

2. **肯德基(学府店)**
   - 地址：丹凤街39号金润发购物中心1层
   - 营业时间：周一至周日 10:00-23:00
   - 电话：025-83222982
   - 评分：4.8
   - 费用：约44元

3. **肯德基(广州店)**
   - 地址：广州路104号
   - 营业时间：24小时营业
   - 电话：025-86632902
   - 评分：4.6
   - 费用：约41元

你可以根据自己的位置和时间选择最合适的肯德基。
